In [1]:
import numpy as np
import math

In [2]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    
    :param a: matrix with dimensions x by y
    :param b: matrix with dimensions y by z
    :return:  matrix with dimensions x by z
    """
    x, y1 = a.shape
    y2, z = b.shape
    
    if y1 != y2:
        raise ValueError('Shapes of matrices is incorrect')
    
    y = y1
    result = np.empty((x, z))
    
    for row in range(x):
        for col in range(z):
            dot_product = 0
            for i in range(y):
                dot_product += a.item((row,i))*b.item((i,col))
            result[row,col] = dot_product
    return result

m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

agh_superfast_matrix_multiply(m1, m2)
res = m1 * m2
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

In [3]:
def gauss_jordan_elimination(A: np.matrix, b: np.matrix) -> np.matrix:
    """Perform gauss jordan elimination for A*x = b problem.
    
    :param A: coefficients matrix with dimensions x by y
    :param b: right hand side column vector with dimensions y 
    """
    
    x,y = A.shape
        
    for col in range(x):
        for row in range(col+1, y):
            scalar = A[row, col]/A[col, col]
            for i in range(col, x):
                A[row, i] -= scalar * A[col, i]
            b[row] -= scalar*b[col]
            
    for col in range(x-1, -1, -1):
        for row in range(col-1, -1, -1):
            scalar = A[row, col]/A[col,col]
            A[row,col] = 0
            b[row] -= scalar* b[col]
            
    for row in range(y):
        b[row] = b[row]/A[row, row]
        
    return b

In [4]:
def gauss_jordan_elimination_with_pivoting(A: np.matrix, b: np.matrix) -> np.matrix:
    """Perform gauss jordan elimination for A*x = b problem, using pivoting.
    
    :param A: coefficients matrix with dimensions x by y
    :param b: right hand side column vector with dimensions y 
    """
    x,y = A.shape
            
    for col in range(x):
        #Find pivot
        pivot = col
        for row in range(col+1, y):
            if(abs(A[row, col]) > abs(A[pivot, col])):
                pivot = row
        # swap
        A[col, :], b[col], A[pivot, :], b[pivot] = A[pivot, :], b[pivot], A[col, :].copy(), b[col].copy()

        for row in range(col+1, y):
            scalar = A[row, col]/A[col, col]
            for i in range(col, x):
                A[row, i] -= scalar * A[col, i]
            b[row] -= scalar*b[col]
            
    for col in range(x-1, -1, -1):
        for row in range(col-1, -1, -1):
            scalar = A[row, col]/A[col,col]
            A[row,col] = 0
            b[row] -= scalar* b[col]
            
    for row in range(y):
        b[row] = b[row]/A[row, row]
        
    return b

In [5]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()
x1 = np.linalg.solve(A, b)
x2 = gauss_jordan_elimination_with_pivoting(A.copy(), b.copy())
x3 = gauss_jordan_elimination(A, b)
np.allclose(x1, x2), np.allclose(x1, x3)

(True, True)

In [6]:
x3 - x1, x3 - x2

(matrix([[-3.88808430e-12],
         [ 4.53486207e-13],
         [-4.53859172e-12],
         [ 3.65774078e-12]]), matrix([[-3.88783450e-12],
         [ 4.53350898e-13],
         [-4.53892479e-12],
         [ 3.65774078e-12]]))

In [7]:
def agh_superfast_lu(a: np.matrix) -> (np.matrix, np.matrix):
    """Perform LU decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    x, y = a.shape
    if(x != y):
        raise ValueError('Shapes of matrix is incorrect')
    lower, upper = np.zeros((x, x)), np.zeros((x, x))
    
    for i in range(x):
        for k in range(i, x):
            sum_tmp = sum(lower[i, j] * upper[j, k] for j in range(i))
            upper[i, k] = a[i, k] - sum_tmp;
            
        for k in range(i, x):
            if(i == k): 
                lower[i, i] = 1
            else:
                sum_tmp = sum(lower[k, j] * upper[j, i] for j in range(i))
                lower[k, i] = (a[k, i] - sum_tmp) / upper[i][i];
    return (lower, upper)
a = np.matrix([[3, -5, 5, 7],
               [2, 1,  1.2, 8],
               [8, 5,  4, 1],
               [6, -2,  2, 3]])  
(l, u) = agh_superfast_lu(a)
np.allclose(np.dot(l, u), a)

True

In [8]:
def agh_superfast_check_spd(a: np.matrix) -> bool:
    """Check whether a matrix is symmetric and positive-definite (SPD).
    
    :param a: _
    """
    if np.allclose(a.transpose(),a) and np.all(np.linalg.eigvals(a)>0): 
        return True
    else:
        return False
    
a = np.matrix([[4, 12, -16],
               [12, 37,  -43],
               [-16, -43,  97]])

agh_superfast_check_spd(a)

True

In [9]:
def agh_superfast_cholesky(a: np.matrix) -> np.matrix:
    """Perform a Cholesky decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    if not agh_superfast_check_spd(a):
        raise ValueError('Matrix must be SPD')
    
    x, y = a.shape
    if(x != y):
        raise ValueError('Shapes of matrix is incorrect')
        
    L = np.zeros((x, x))
    for i in range(x):
        for k in range(i+1):
            sum_tmp = sum(L[i, j] * L[k, j] for j in range(k))
            
            if(k == i):
                L[i, k] = math.sqrt(a[i, i] - sum_tmp)
            else:
                L[i, k] = (1.0 / L[k, k] * (a[i, k] - sum_tmp))
    return L
a = np.matrix([[4, 12, -16],
               [12, 37,  -43],
               [-16, -43,  97]])
L = agh_superfast_cholesky(a)
np.allclose(a, np.dot(L, L.conj().transpose()))

True